In [6]:

from collections import namedtuple
from PIL import Image, ImageDraw, ImageFont
from google.cloud import vision
from enum import Enum
%matplotlib inline
import re
import matplotlib.pyplot as plt
import os
import json
from google.protobuf.json_format import MessageToJson
from google.protobuf import json_format
import requests
import sys
sys.path.append("..")
from models.bounding_box import FeatureType, Point, BoundingBox
from utils.cv_preprocess import draw_boxes, group_bounding_boxes, merge_box_groups

from utils.file_utils import *

from utils.nlp_preprocess import load_spacy_models, is_english, is_chinese, split_dish_info, is_word_relevant

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../menu-ocr-390814-027d51b70720.json'


In [7]:
# Load the existing JSON file at the start
filename = 'trip_advisor_london.json'
pdf_img_directory = "../downloaded_menu/converted/"
if os.path.exists(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        all_shop_data = json.load(f)

In [8]:
save_directory = "../downloaded_menu/raw_file"
pdf_img_directory = "../downloaded_menu/cleaned_img/"
create_dir(save_directory)
create_dir(pdf_img_directory)

In [9]:
docx_pattern = re.compile(r".*\.docx")
pdf_pattern = re.compile(r".*\.pdf")
image_pattern = re.compile(r".*\.(jpg|jpeg|png)")


In [10]:
flag_stop = False
for restaurant in all_shop_data:
    if flag_stop:
        print("Last review reached")
        break
    menu_count = 0
    restaurant_name = restaurant['name'].replace(' ', '_').replace('/', '_')  # Replace spaces with underscores and handle potential invalid characters like '/'
    if 'last_reviewed' in restaurant and restaurant['last_reviewed']:
        flag_stop = True
    if 'pure_english' in restaurant and restaurant['pure_english']:
        continue
    
        
        
    for menu_url in restaurant["menu_src"]:
        if not (docx_pattern.match(menu_url) or pdf_pattern.match(menu_url) or image_pattern.match(menu_url)):
            print(f"Skipping {menu_url} due to unsupported file type.")
            continue
        
        # Generate a filename based on the restaurant name and counter
        file_extension = os.path.splitext(menu_url)[1]  # Extract the file extension from the menu URL
        filename = f"{restaurant_name}_{menu_count}{file_extension}"
        
        # Increase the menu counter
        menu_count += 1

        # Local path to save the file
        if image_pattern.match(menu_url):
            save_path = os.path.join(pdf_img_directory, filename)
        else:
            save_path = os.path.join(save_directory, filename)

        # Check if the file exists remotely
        response = requests.head(menu_url)
        if response.status_code == 200:
            if not os.path.exists(save_path):
                # If the file doesn't exist locally, download it
                with requests.get(menu_url, stream=True) as r:
                    with open(save_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                print(f"Downloaded: {filename}")
                if pdf_pattern.match(menu_url):
                    convert_pdf_to_img(save_path, pdf_img_directory)
            else:
                print(f"File {filename} already exists locally.")
        else:
            print(f"File {filename} not found remotely.")

print("Process completed.")

Downloaded: 1._Gouqi_London_0.pdf
Page 1 has been processed and saved as 1._Gouqi_London_0_page_1.png
Page 2 has been processed and saved as 1._Gouqi_London_0_page_2.png
Page 3 has been processed and saved as 1._Gouqi_London_0_page_3.png
Page 4 has been processed and saved as 1._Gouqi_London_0_page_4.png
Page 5 has been processed and saved as 1._Gouqi_London_0_page_5.png
Page 6 has been processed and saved as 1._Gouqi_London_0_page_6.png
Page 7 has been processed and saved as 1._Gouqi_London_0_page_7.png
Page 8 has been processed and saved as 1._Gouqi_London_0_page_8.png
Page 9 has been processed and saved as 1._Gouqi_London_0_page_9.png
Page 10 has been processed and saved as 1._Gouqi_London_0_page_10.png
Page 11 has been processed and saved as 1._Gouqi_London_0_page_11.png
Page 12 has been processed and saved as 1._Gouqi_London_0_page_12.png
Page 13 has been processed and saved as 1._Gouqi_London_0_page_13.png
Page 14 has been processed and saved as 1._Gouqi_London_0_page_14.png
Down

TypeError: expected string or bytes-like object